In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
%matplotlib inline

In [2]:
# Parameters
smote = SMOTE(random_state=42)

In [35]:
dataset = pd.read_csv(filepath_or_buffer='C:/Users/jvsn1/Documents/TRN', sep='\t', index_col = 0)

In [33]:
dataset.head()

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1,IND_BOM_1_2
INDEX,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,0,1,1,0,1,1,1,0,1
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,0,1,0,1,1,0,0,0,1,0
2,1,0,1,0,0,1,0,0.281910,0,1.000000,...,1,1,0,0,0,0,1,0,1,0
3,1,1,1,0,0,0,0,0.225741,0,0.111111,...,1,1,0,1,1,0,1,0,1,0
4,1,1,0,0,0,1,0,0.480403,0,0.111111,...,1,1,1,0,0,1,0,1,1,0


In [38]:
y = dataset['IND_BOM_1_1']
X = dataset.drop(columns=['IND_BOM_1_1', 'IND_BOM_1_2'])

<h1>Buscando Atributos Categóricos</h1>
<p>É possível encontrar atributos categóricos buscando àqueles que possuem valores binários (0 ou 1). É importante identificar esses atributos e retirá-los temporariamente para rodar o SMOTE</p>

In [43]:
categorical = []
for column in X:
    if len(X[column].unique()) == 2:
        categorical.append(column)

In [54]:
print('Número de atributos categóricos: %d'%(len(categorical)))

Número de atributos categóricos: 99


In [61]:
categorical_mean = X[categorical].mean()
categorical_mean.sort_values(ascending=False)[:20]

RENDA_VIZINHANCA_2            0.967906
SIGLA_PARTIDO_FILIADO_1       0.964923
COMPARATIVO_RENDA_CEP_1       0.963034
BANCO_REST_IRPF_ULTIMA_6      0.926109
RENDA_VIZINHANCA_4            0.922923
BANCO_REST_IRPF_ULTIMA_7      0.919508
CLASSE_SOCIAL_CONSUMIDOR_2    0.917957
SIGLA_PARTIDO_FILIADO_2       0.892833
UF_1                          0.889274
SIGLA_PARTIDO_FILIADO_3       0.852432
FLAG_REDE_SOCIAL_1            0.809186
CEP4_1                        0.806082
CEP2_1                        0.791557
CLASSE_SOCIAL_CONSUMIDOR_4    0.789284
CEP3_1                        0.750403
UF_2                          0.691952
CEP4_2                        0.691636
CEP3_2                        0.607314
CEP4_3                        0.598113
CEP2_2                        0.587570
dtype: float64

<h2>Classes (IND_BOM_1_1 and IND_BOM_1_2)</h2>
<p>Here we can join both columns and define IND_BOM_1_1 as 1 and IND_BOM_1_2 as 0. For that we just need to drop the column  IND_BOM_1_1.</p>

In [36]:
X = dataset.drop(columns = ['IND_BOM_1_1'])
y = dataset['IND_BOM_1_1']

In [37]:
pca = PCA()

In [38]:
features = list(X)

<h2>Dividir o Conjunto em Treinamento e Teste: K-Fold</h2>
<p>K-fold divide o dataset em k conjuntos de treinamento e teste independente da distribuição das classes, ou seja, ainda é necessário executar o SMOTE em cada conjunto</p>

In [65]:
skf = KFold(n_splits=5)
file_folds = open('./aux_files/test_indexes.txt', 'w')
file_index = 1
for train_index, test_index in skf.split(X, y):
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    file_folds.write('#' + str(file_index) + ': ' + ' '.join(map(str, test_index)) + '\n')
    X_train, y_train = runSmote(train_index)
    X_train.to_csv('./aux_files/X_train' + str(file_index) + '.csv')
    y_train.to_csv('./aux_files/y_train' + str(file_index) + '.csv')
    file_index += 1

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [64]:
def runSmote(train_index):
    b_X_train, b_y_train = sm.fit_sample(X.iloc[train_index], y.iloc[train_index])
    return b_X_train, b_y_train

In [39]:
dataset.cov

<bound method DataFrame.cov of         UF_1  UF_2  UF_3  UF_4  UF_5  UF_6  UF_7         IDADE  SEXO_1  \
INDEX                                                                    
0          1     1     1     0     0     0     0  1.350979e-01       1   
1          1     0     1     0     0     1     0  2.735041e-01       1   
2          1     0     1     0     0     1     0  2.819095e-01       0   
3          1     1     1     0     0     0     0  2.257406e-01       0   
4          1     1     0     0     0     1     0  4.804034e-01       0   
5          0     1     1     0     0     0     1  2.193229e-01       0   
6          1     1     1     0     0     0     0  5.734882e-01       0   
7          0     1     0     0     1     0     1  8.343982e-01       1   
8          1     1     0     1     0     0     0  3.474494e-01       1   
9          1     1     1     0     0     0     0  5.506237e-16       1   
10         1     0     1     1     0     0     0  6.547031e-01       0   
11     

In [48]:
correlation_matrix = dataset.corr()

In [74]:
correlation_matrix['IND_BOM_1_1'][(correlation_matrix['IND_BOM_1_1'] > 0.1) & (correlation_matrix['IND_BOM_1_1'] < 1.0)]

IDADE                      0.149692
ATUALIZACAO_ENDERECO       0.116064
COMPARATIVO_RENDA_CEP_3    0.153344
Name: IND_BOM_1_1, dtype: float64